In [1]:
# easy_install pyscreenshot

In [11]:
import pyscreenshot
import pyautogui, sys
import operator
import math
import numpy as np
from PIL import Image
import time
from pynput import mouse
from pynput.mouse import Listener, Button, Controller

p = (295, 179)
cell_width = 16
cell_hight = 16.0
cell_trained_size = (24,24)
cell_dimentions = (30,16)
cell_margine = 3 #I use this margine get into the core cell
cell_upper_margine = 1
cell_margine_2 = 6 #I use this margine get into the core cell
cell_upper_margine_2 = 2

cell_list = []
cell_names = {0:'one', 1:'two', 2:'three', 3:'four', 4:'five', 5:'six', 6:'empty'
              , 7:'covered', 8:'marked', 9:'blow', 10:'bad', 11:'bad_1'}
cell_numbers = {'one' : 0, 'two':1, 'three':2, 'four':3, 'five':4, 'six':5, 'empty':6
              , 'covered':7, 'marked':8, 'blow':9, 'bad':10, 'bad_1':11}

mouse_both_click = 'both'
mouse_right_click = 'right'
mouse_left_click = 'left'

pyautogui.FAILSAFE = False
pyautogui.PAUSE = 0

In [12]:
def init_trained_cell(size):
    global cell_list
    i1 = list(Image.open("cell_one.png").resize(size).getdata())
    #Image.open("cell_one.png").resize(size).show()
    i2 = list(Image.open("cell_two.png").resize(size).getdata())
    i3 = list(Image.open("cell_three.png").resize(size).getdata())
    #Image.open("cell_three.png").resize(size).show()
    i4 = list(Image.open("cell_four.png").resize(size).getdata())
    i5 = list(Image.open("cell_five.png").resize(size).getdata())
    i6 = list(Image.open("cell_six.png").resize(size).getdata())
    e_cell = list(Image.open("cell_empty.png").resize(size).getdata())
    c_cell = list(Image.open("cell_covered.png").resize(size).getdata())
    m_cell = list(Image.open("cell_marked.png").resize(size).getdata())
    blow_cell = list(Image.open("cell_blow.png").resize(size).getdata())
    bad_cell = list(Image.open("cell_bad.png").resize(size).getdata())
    bad_1_cell = list(Image.open("cell_bad_1.png").resize(size).getdata())
    cell_list = [i1,i2,i3,i4,i5,i6,e_cell,c_cell,
                 m_cell, blow_cell, bad_cell, bad_1_cell]
def insight_training():
    print('starting training cell insight')
    global_data = []
    for g_data in cell_list:
        distinct_data = {}
        init_trained_cell((24,24))
        for i in g_data:
            if(i not in distinct_data.keys()):
                distinct_data[i] = 0
            else:
                distinct_data[i] = distinct_data.get(i)+1
        global_data.append(distinct_data)    
    counter = 0
    for i in global_data:
        print(cell_names.get(counter))
        counter +=1
        for key, value in i.items():
            if(value > 40):
                print(key, ' -- ', value)
        print('----------------')
#insight_training()

starting training cell insight
----------------
### one
(144, 151, 204, 255)  --  43  
(55, 74, 235, 255)  --  43  
(11, 36, 251, 255)  --  87  

### two
(145, 172, 146, 255)  --  75  
(58, 139, 60, 255)  --  75  
(14, 122, 17, 255)  --  127 

### three
(205, 145, 148, 255)  --  87  
(236, 57, 67, 255)  --  87  
(252, 13, 27, 255)  --  123  

### four
(142, 144, 172, 255)  --  59  
(49, 55, 138, 255)  --  59  
(2, 11, 121, 255)  --  127  

### five
(172, 142, 143, 255)  --  103  
(139, 49, 52, 255)  --  103  
(122, 3, 7, 255)  --  147  

### six
(146, 172, 172, 255)  --  87  
(59, 139, 139, 255)  --  87  
(16, 123, 122, 255)  --  159  

### empty
(189, 189, 189, 255)  --  575  

### covered
(205, 205, 205, 255)  --  43  
(172, 172, 172, 255)  --  43  

### marked
(205, 205, 205, 255)  --  43  
(172, 172, 172, 255)  --  43  

### blow
(154, 154, 154, 255)  --  43  
(0, 0, 0, 255)  --  171  

### bad
(0, 0, 0, 255)  --  59  
(205, 11, 22, 255)  --  41  

### bad_1
(252, 13, 27, 255)  --  143  
(205, 11, 22, 255)  --  43  
(0, 0, 0, 255)  --  171  


In [20]:
def cSub(p1, p2):
    r,g,b,a = p1
    x,y,z,s = p2
    d1 = (abs(r-x) )
    d2 = (abs(g-y))
    d3 = (abs(b-z))
    return (d1+d2+d3)

def calculateDiff(a,b):
    return sum(list(map(cSub, a, b)))
def findBestMatch(a):
    diff = []
    #diff.append(calculateDiff(a, i4))
    if(calculateDiff(a, cell_list[cell_numbers.get('covered')]) < 15):
        return cell_numbers.get('covered')
    if(calculateDiff(a, cell_list[cell_numbers.get('empty')]) < 15):
        return cell_numbers.get('empty')
    for cell in cell_list:
        cost = calculateDiff(a, cell)
        diff.append(cost)
        if(cost < 15): 
            break
    return(diff.index(min(diff)))
def getCellImg(x,y):
    'this method is very slow takes about .2 second per cell, which does not make sense to work with it'
    return pyscreenshot.grab(bbox=(p[0] + cell_upper_margine + (x*cell_width),
                                   p[1] + cell_upper_margine + (y*cell_width),
                                   p[0] - cell_margine + ((x+1)*cell_width),
                                   p[1] - cell_margine + ((y+1)*cell_hight) ))

def getCellImgFromImage_modified(img,x,y, c_width = cell_width):
    'this method is much better performance .02 seconds for each cell'
    return img.crop(((x*c_width)+2,
                                   (y*c_width)+3,
                                   ((x+1)*c_width) ,
                                   ((y+1)*c_width)  ))

def getCellImgFromImage(img,x,y, c_width = cell_width):
    'this method is much better performance .02 seconds for each cell'
    return img.crop((cell_upper_margine_2 + (x*c_width*2),
                                   cell_upper_margine_2 + (y*c_width*2),
                                   ((x+1)*c_width*2) - cell_margine_2,
                                   ((y+1)*c_width*2) - cell_margine_2 ))


def getFullImg():
    return pyscreenshot.grab(bbox=(p[0] ,
                                   p[1] ,
                                   p[0] + (cell_width * cell_dimentions[0]),
                                   p[1] + (cell_hight * cell_dimentions[1]) ))
def getFullImgWithCoordinations(x,y, c_width = cell_width):
    #print(c_width)
    return pyscreenshot.grab(bbox=(x ,
                                   y ,
                                   x + (c_width * cell_dimentions[0]),
                                   y + (c_width * cell_dimentions[1]) ))

def moveMouseToCell_modified(x,y, cell_x, cell_y, c_width):
    print(x+(cell_x*c_width) + 3)
    print(y+(cell_y*c_width)+2,4)
    pyautogui.moveTo(x+(cell_x*c_width) + 3, y+(cell_y*c_width)+2,4)

def moveMouseToCell(x,y, cell_x, cell_y, c_width):
    pyautogui.moveTo(x+(cell_x*c_width) + cell_upper_margine_2 + 3, y+(cell_y*c_width)+cell_margine+2)

def mouseAction(action, x, y, cell_x, cell_y):
    if(action == mouse_right_click):
        pyautogui.mouseDown(x=None, y=None, button='right')
        pyautogui.mouseUp(x=None, y=None, button='right')
    elif(action == mouse_both_click):
        pyautogui.mouseDown(x=None, y=None, button='left')
        pyautogui.mouseDown(x=None, y=None, button='right')
        pyautogui.mouseUp(x=None, y=None, button='left')
        pyautogui.mouseUp(x=None, y=None, button='right')

In [14]:
def initiateCells():
    getCellImg(0,0).save('cell_one.png')
    getCellImg(0,1).save('cell_two.png')
    getCellImg(1,1).save('cell_three.png')
    getCellImg(9,1).save('cell_four.png')
    getCellImg(24,7).save('cell_five.png')
    getCellImg(3,4).save('cell_empty.png')
    getCellImg(16,1).save('cell_covered.png')
    getCellImg(1,0).save('cell_marked.png')
    getCellImg(16,0).save('cell_blow.png')
    getCellImg(9,0).save('cell_bad.png')
    getCellImg(10,0).save('cell_bad_1.png')
    return None
def initiateCellsWithImage():
    im = Image.open('training.png')
    #im.show()
    getCellImgFromImage(im,0,0).save('tf/cell_one.png')
    getCellImgFromImage(im,0,1).save('tf/cell_two.png')
    getCellImgFromImage(im,1,1).save('tf/cell_three.png')
    getCellImgFromImage(im,9,1).save('tf/cell_four.png')
    getCellImgFromImage(im,24,7).save('tf/cell_five.png')
    getCellImgFromImage(im,3,4).save('tf/cell_empty.png')
    getCellImgFromImage(im,16,1).save('tf/cell_covered.png')
    getCellImgFromImage(im,1,0).save('tf/cell_marked.png')
    getCellImgFromImage(im,16,0).save('tf/cell_blow.png')
    getCellImgFromImage(im,9,0).save('tf/cell_bad.png')
    getCellImgFromImage(im,10,0).save('tf/cell_bad_1.png')
    im = Image.open('training_1.png')
    getCellImgFromImage(im,9,6).save('tf/cell_six.png')    
    return None
#initiateCellsWithImage()
#initiateCells() #it is danger to use this function

In [15]:
def manipulate_cell_matrix(fileName, c_width = cell_width):
    #print(fileName)
    cell_matrix = []
    im = Image.open(fileName)
    #im.show()
    for i in range(cell_dimentions[0]):
        cell_matrix.append([])
        for j in range(cell_dimentions[1]):#cell_dimentions[1]):
            #if((i == 0 and j < 3) or (i == 9 and j < 7)):
                #getCellImgFromImage(im,i,j, c_width).show()
            ic = list(getCellImgFromImage(im,i,j, c_width).getdata())
            #print('manipulate -- ', i, ' - ', j)
            cell_matrix[i].append((findBestMatch(ic)))
    return np.transpose(cell_matrix)

In [16]:
def list_surrounded_index(matrix,x,y):
    index = []
    for i in range(x-1, x+2):
        if i<0 or i+1>cell_dimentions[1]:
            continue
        for j in range(y-1, y+2):
            if j<0 or j+1>cell_dimentions[0]:
                continue
            index.append((i,j))
    return index
def verify_bouble_click(matrix, x,y):
    for i,j in list_surrounded_index(matrix,x,y):
        if(matrix[i][j] == cell_numbers.get('covered')):
                return True
    return False

def get_double_click_action(matrix, x, y):
    dca = []
    for i,j in list_surrounded_index(matrix,x,y):
        if(matrix[i][j] < cell_numbers.get('empty')):
                dca.append((mouse_both_click, (i,j)))
    return dca

def investigate_action(matrix, x, y):
    actions = []
    ni = np.zeros(9).reshape(3,3)
    covered = []
    marked = []
    for i,j in list_surrounded_index(matrix,x,y):
            if(matrix[i][j] == cell_numbers.get('covered')):
                covered.append((i,j))
            if(matrix[i][j] == cell_numbers.get('marked')):
                marked.append((i,j))
    if(len(marked) + len(covered) == matrix[x][y]+1):
        for i in covered:
            actions.append((mouse_right_click, i))
    return actions
            
def get_actions(matrix):
    global_actions = []
    double_click_actions = []
    r_actions = False
    for i in range(cell_dimentions[1]):
        for j in range(cell_dimentions[0]):
            if(matrix[i][j]>cell_numbers.get('marked')):
                return [], False
            if(matrix[i][j] == cell_numbers.get('marked')):
                for a2 in get_double_click_action(matrix, i, j):
                    if(a2 in double_click_actions):
                        pass
                    else:
                        double_click_actions.append(a2)
            if(matrix[i][j] < cell_numbers.get('empty')):
                actions = investigate_action(matrix, i, j)
                if(len(actions) > 0):
                    for a in actions:
                        if(a in global_actions):
                            pass
                        else:
                            global_actions.append(a)
                            dca = get_double_click_action(matrix, a[1][0], a[1][1])
                            for a2 in dca:
                                if(a2 in double_click_actions):
                                    pass
                                else:
                                    double_click_actions.append(a2)
    if(len(global_actions) > 0):
        r_actions = True
    for a in double_click_actions:
        if(verify_bouble_click(matrix, a[1][0], a[1][1])):
            global_actions.append(a)
    return global_actions, r_actions

def execute_actions(x, y, actions, c_width = cell_width):
    for a in actions:
        action, cord = a
        #print(a)
        moveMouseToCell(x,y,cord[1], cord[0], c_width)
        mouseAction(action,x,y, cord[1], cord[0])

In [19]:
def play(first_click, second_click):
    mousex, mousey = first_click
    c_width = int(round(abs(second_click[0] - first_click[0])/cell_dimentions[0]))
    #print(c_width)
    t = "snapshots/" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime()) + '.png'
    getFullImgWithCoordinations(mousex, mousey, c_width).save(t)
    im = Image.open(t)
    train_cell = getCellImgFromImage(im, 0,0)
    #print(train_cell.size)
    #train_cell.show()
    init_trained_cell(train_cell.size)
    count = 0
    action_list=[]
    snaps_lis=[]
    while(True):
        t = "snapshots/" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime()) + '.png'
        getFullImgWithCoordinations(mousex, mousey, c_width).save(t)
        matrix = manipulate_cell_matrix(t, c_width)
        #print(matrix)
        actions, ra = get_actions(matrix)
        if(ra):
            count = 0
        else:
            count += 1
        if(len(actions) < 1 or count > 1):
            actions = []
            s = input('continue? y/n')
            if(s == 'n'):
                break
        execute_actions(mousex, mousey, actions, c_width)
        #store_actions(action_list,snaps_list,actions,t)

In [ ]:
def main_function():
    num_clicks = 0
    first_click = 0 
    second_click =0
    def on_click(x, y, button, pressed):
        nonlocal first_click, second_click, num_clicks
        #print('{0} at {1}'.format(
            #'Pressed' if pressed else 'Released',
            #(x, y)))
        if not pressed:
            num_clicks += 1
            #print(num_clicks)
            if(num_clicks == 2):
                second_click = (x,y)
                # Stop listener
                play(first_click, second_click)
                return False
            if(num_clicks == 1):
                first_click = (x,y)
    
        # Collect events until released
    with Listener(
        on_click=on_click,
    ) as listener:
        listener.join()
main_function()

In [90]:
from pynput import mouse
from pynput.mouse import Listener

#pyautogui.moveTo(100, 200, duration=3)
print(dir(mouse.Listener))
def on_move(x, y):
    print('Pointer moved to {0}'.format(
        (x, y)))

def on_click(x, y, button, pressed):
    print('{0} at {1}'.format(
        'Pressed' if pressed else 'Released',
        (x, y)))
    if not pressed:
        # Stop listener
        return False

def on_scroll(x, y, dx, dy):
    print('Scrolled {0}'.format(
        (x, y)))

# Collect events until released
with Listener(
        #on_move=on_move,
        on_click=on_click,
        #on_scroll=on_scroll
        ) as listener:
    listener.join()

#print('starting')
#print(cell_dimentions)
#for i in range(cell_dimentions[0]):
#    cell_matrix.append([])
#    print(i)
#    for j in range(cell_dimentions[1]):
#        ic = list(getCellImg(j,i).getdata())
#        cell_matrix[i].append(cell_names.get(findBestMatch(ic)))'''
#print(dir(pyscreenshot.loader))
#print(pyscreenshot.about)
#im = Image.open('training.png')
#cell_matrix = []
#for i in range(cell_dimentions[0]):
#    cell_matrix.append([])
#    for j in range(cell_dimentions[1]):#cell_dimentions[1]):
#        ic = list(getCellImgFromImage(im,i,j).getdata())
#        #cell_matrix[i].append(cell_names.get(findBestMatch(ic)))
#        cell_matrix[i].append((findBestMatch(ic)))
#
#cell_matrix = np.transpose(cell_matrix)
#for i in range(cell_dimentions[1]):
#    print(cell_matrix[i])
#    print('------------')
#fcell = getCellImgFromImage(im, 28, 14)
#print(cell_names.get(findBestMatch(fcell.getdata())))
#imc = im.crop((0,0,32,32))
#imc.show()
#im.show()
#im.show()
#print(dir(im))
#im2 = Image.frombytes('RGBA',(960,512),im.tobytes())
#im2.show()
#np_frame = np.array(im_frame.getdata())
#print(cell_matrix)
#fi = getFullImg()
#fi.show()
#print(dir(fi))
#b = fi.getpixel
#print((b))
#b = fi.getpixel((0,0))
#print(b)
#b = fi.getpixel((511,959))
#print(b[0])
#getCellImg(29,15).show()
#ic = list(getCellImg(29,0).getdata())
#print(cell_names.get(findBestMatch(ic)))
#getCellImg(29,15).show()

# part of the screen
#sh=pyscreenshot.grab(bbox=(10,10,11,12))
#screenshot.show()
#print((sh.getpixel((0,0))))
#print(sh.size)

# save to file
#fi.save('training.png')

['StopException', '_EVENTS', '_HANDLED_EXCEPTIONS', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_bootstrap', '_bootstrap_inner', '_create_event_tap', '_delete', '_emitter', '_exc_info', '_handle', '_handler', '_initialized', '_mark_ready', '_reset_internal_locks', '_run', '_set_ident', '_set_tstate_lock', '_stop', '_wait_for_tstate_lock', 'daemon', 'getName', 'ident', 'isAlive', 'isDaemon', 'is_alive', 'join', 'name', 'run', 'running', 'setDaemon', 'setName', 'start', 'stop', 'wait']
Pressed at (295.1015625, 179.80078125)
Released at (295.1015625, 179.80078125)


In [74]:
im = Image.open('snapshots/2018-02-23-16-35-46.png')
ic = (getCellImgFromImage(im,0,0,32))
print(dir(ic))
print(ic.size)
ic1 = ic.resize((24,24))
print(ic.size)
print(ic1.size)
ic1.show()

['_Image__transformer', '__array_interface__', '__class__', '__copy__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_close_exclusive_fp_after_loading', '_copy', '_crop', '_dump', '_expand', '_new', '_repr_png_', 'alpha_composite', 'category', 'close', 'convert', 'copy', 'crop', 'draft', 'effect_spread', 'filter', 'format', 'format_description', 'frombytes', 'fromstring', 'getbands', 'getbbox', 'getcolors', 'getdata', 'getextrema', 'getim', 'getpalette', 'getpixel', 'getprojection', 'height', 'histogram', 'im', 'info', 'load', 'mode', 'offset', 'palette', 'paste', 'point', 'putalpha', 'putdata', 'putpalette', 'putpixel', 'pyaccess', 'qua